<a href="https://colab.research.google.com/github/dp-93/Train-YOLO-model-to-detect-cards/blob/main/Train_Custom_YOLOv8_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# --- 1. Install Ultralytics and Roboflow ---
!pip install -q ultralytics roboflow

from ultralytics import YOLO
from roboflow import Roboflow
import os

# --- 2. Authenticate with Roboflow and Download Dataset ---
# IMPORTANT: Make sure you have your free API key from https://roboflow.com/
# It looks like your key is "uifQd4NPxTowFlX0Zfrc", but always keep keys private.
ROBOFLOW_API_KEY = "uifQd4NPxTowFlX0Zfrc" # Replace with your key

# This is the new, correct code snippet you generated from the Roboflow website.
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("train-yolo-8zb0f").project("playing-cards-ow27d-4nsnm")
version = project.version(1)
dataset = version.download("yolov8")
# ---------------------------------------------


# --- 3. Define the Path to the Dataset Configuration File ---
# This part remains the same. It finds the data.yaml file inside the downloaded dataset.
data_yaml_path = os.path.join(dataset.location, "data.yaml")


# --- 4. Train (Fine-Tune) the YOLOv8 Model ---
# This part also remains the same.
print("\nStarting the training process...")
model = YOLO('yolov8n.pt')

# *** THE CHANGE IS HERE: Reduced epochs from 50 to 15 for faster training ***
results = model.train(data=data_yaml_path, epochs=15, imgsz=640)

print("\n--- Training Complete! ---")
print("Your custom model has been trained and the best version is saved as 'best.pt'.")
print("You can find it in the folder: /content/runs/detect/train/weights/best.pt")



loading Roboflow workspace...
loading Roboflow project...

Starting the training process...
Ultralytics 8.3.199 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Playing-Cards-1/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, o

In [3]:
# --- 1. Install Libraries ---
!pip install -q ultralytics gradio

import gradio as gr
from ultralytics import YOLO
import torch

# --- 2. Upload Your Custom Model ---
# This cell will prompt you to upload the 'best.pt' file you downloaded
# from your training run.
from google.colab import files
import os

print("Please upload your custom-trained 'best.pt' model file.")
uploaded = files.upload()

# Check if the file was uploaded and set the model path
if 'best.pt' in uploaded:
    model_path = 'best.pt'
    print(f"Model '{model_path}' uploaded successfully.")

    # --- 3. Load Your Custom-Trained Model ---
    # Load the model from the uploaded file
    model = YOLO(model_path)
    print("Custom YOLOv8 model loaded successfully!")

    # --- 4. Define the Prediction Function ---
    # This function takes an image and returns the image with detections drawn on it.
    def detect_cards(image):
        # Run inference on the image
        results = model(image)

        # Plot the results on the image
        # The render() method returns a NumPy array of the image with boxes and labels
        annotated_image = results[0].plot()

        # Convert from BGR (OpenCV format) to RGB (PIL/Gradio format)
        annotated_image_rgb = annotated_image[..., ::-1]

        return annotated_image_rgb

    # --- 5. Create and Launch the Gradio Interface ---
    interface = gr.Interface(
        fn=detect_cards,
        inputs=gr.Image(type="numpy", label="Upload an image with playing cards"),
        outputs=gr.Image(type="numpy", label="Detection Result"),
        title="🃏 Custom Playing Card Detector",
        description="This app uses a custom-trained YOLOv8 model to detect and identify playing cards in an image. Upload an image to see it in action!"
    )

    interface.launch(debug=True)

else:
    print("\nError: 'best.pt' not found. Please make sure you upload the file with the correct name.")


Please upload your custom-trained 'best.pt' model file.


Saving best.pt to best.pt
Model 'best.pt' uploaded successfully.
Custom YOLOv8 model loaded successfully!
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b85f0756a303156bc4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



0: 640x640 (no detections), 9.0ms
Speed: 2.9ms preprocess, 9.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 45.0ms
Speed: 3.2ms preprocess, 45.0ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b85f0756a303156bc4.gradio.live
